In [2]:
import pandas as pd
import requests

In [ ]:
# REGIONALES (candidatos 267 listas)
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('regionales.csv')

In [ ]:
# PROVINCIALES (candidatos 1562)
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('provinciales.csv')

In [ ]:
# DISTRITALES
# primero 19
# ultimo 16860
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('distritales.csv')

In [ ]:
# LISTA DE CANDIDATOS

for txCodExpedienteExt in df_provinciales['txCodExpedienteExt']:
    url = f'https://apiplataformaelectoral3.jne.gob.pe/api/v1/plan-gobierno/candidatos?IdProcesoElectoral=113&TxCodExpedienteExt={txCodExpedienteExt}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df['txCodExpedienteExt'] = txCodExpedienteExt
        df.to_csv(f"listas_provinciales/{txCodExpedienteExt}.csv")

In [ ]:
# CONCATENAR
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join("listas_provinciales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df_candidatos_provincial = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
df_candidatos_provincial.to_csv('candidatos_provincial.csv')
df_candidatos_provincial

In [ ]:
# LIMPIEZA - verificado
df_partidos_provincial = pd.read_csv('partidos_provincial.csv')
df_candidatos_provincial_limpio = df_candidatos_provincial[df_candidatos_provincial.txCodExpedienteExt.isin(df_partidos_provincial.txCodExpedienteExt)]
df_candidatos_provincial_limpio.to_csv('candidatos_provincial_limpio.csv')

In [ ]:
# ELIMINADOS - verificado
df_candidatos_provincial_eliminados = df_candidatos_provincial[~df_candidatos_provincial.txCodExpedienteExt.isin(df_partidos_provincial.txCodExpedienteExt)]
df_candidatos_provincial_eliminados.to_csv('candidatos_provincial_eliminados.csv')

In [ ]:
# MERGE
df1 = pd.read_csv('candidatos_provincial.csv')
df2 = pd.read_csv('partidos_provincial.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_provincial_final.csv')

In [16]:
# solo gobernadores y vice en regionales y limpiar duplicados
df_gobernadores_vicegobernadores_regional = pd.read_csv('base_datos_regional_final.csv')
df_gobernadores_vicegobernadores_regional = df_gobernadores_vicegobernadores_regional[df_gobernadores_vicegobernadores_regional['cargo'].isin(["[\'GOBERNADOR REGIONAL\']", "[\'VICEGOBERNADOR REGIONAL\']"])].drop(columns=['Unnamed: 0'])
df_gobernadores_vicegobernadores_regional.to_csv('solo_gobernadores_y_vicegobernadores.csv', index=False)

In [ ]:
# Numero de partidos: 147
df2 = pd.read_csv('partidos_general.csv')
numero_partidos = df2['txOrganizacionPolitica'].nunique()
numero_partidos

In [ ]:
# Numero de apellidos unicos: 107330 de 135056 DNIs
df3 = pd.read_csv('base_datos_final.csv')
df3['Apellidos'] = (df3['txNombreCompleto'].str.split().str[-2:].str.join(' '))
personas_unicas = df3['txDocumentoIdentidad'].nunique()
apellidos_unicos = df3['Apellidos'].nunique()
df3.to_csv('base_datos_final.csv')

In [ ]:
# DNI duplicados: 369 registros, 184 personas
dni_duplicados = df3[df3['txDocumentoIdentidad'].duplicated(keep=False)]
dni_duplicados.to_csv('dni_duplicados.csv')
personas_unicas = dni_duplicados['txDocumentoIdentidad'].nunique()
personas_unicas
moda = dni_duplicados['txDocumentoIdentidad'].mode()
moda

In [ ]:
# Elecciones regionales: gobernadores y vice gobernadores
df_regionales_gobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_gobernadores = df_regionales_gobernadores[df_regionales_gobernadores['txCargoEleccion']=='GOBERNADOR REGIONAL'].reset_index()
df_regionales_gobernadores.to_csv('gobernadores_regionales.csv')
df_regionales_vicegobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_vicegobernadores = df_regionales_vicegobernadores[df_regionales_vicegobernadores['txCargoEleccion']=='VICEGOBERNADOR REGIONAL'].reset_index()
df_regionales_vicegobernadores.to_csv('vicegobernadores_regionales.csv')

In [ ]:
# Elecciones provinciales: alcaldes
df_provinciales_alcaldes = pd.read_csv('base_datos_final.csv')
df_provinciales_alcaldes = df_provinciales_alcaldes[df_provinciales_alcaldes['txCargoEleccion']=='ALCALDE PROVINCIAL'].reset_index()
df_provinciales_alcaldes.to_csv('alcaldes_provinciales.csv')